In [ ]:
!pip install git+https://github.com/sladkovm/strava-swagger-client

API initialization code

In [98]:

#from __future__ import print_statement
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

configuration = swagger_client.Configuration()
configuration.access_token = '39915a0dab17e689a6056465807fc83fc505b1f9'

# create an instance of the API class
api_instance = swagger_client.SegmentsApi(swagger_client.ApiClient(configuration))


In [17]:
import math
def get_bounds_in_area(starting_lat,starting_long,distance):
    # returns southwest corner and northeast corner
  top_right =   MaxLatLongOnBearing(starting_lat, starting_long,45,distance)
  bottom_right = MaxLatLongOnBearing(starting_lat, starting_long, 135, distance)
  bottom_left = MaxLatLongOnBearing(starting_lat, starting_long, 225, distance)
  top_left = MaxLatLongOnBearing(starting_lat, starting_long, 315, distance)
  return {"top right":top_right,"bottom right":bottom_right,"bottom left":bottom_left,"top left" :top_left}

def MaxLatLongOnBearing(centerLat, centerLong, bearing, distance):
    

        lon_rads = math.radians(centerLong)
        lat_rads = math.radians(centerLat)
        bearing_rads = math.radians(bearing)
        max_lat_rads = math.asin(math.sin(lat_rads) * math.cos(distance / 6371) + math.cos(lat_rads) * math.sin(distance / 6371) * math.cos(bearing_rads))
        max_lon_rads = lon_rads + math.atan2((math.sin(bearing_rads) * math.sin(distance / 6371) * math.cos(lat_rads)), (math.cos(distance / 6371) - math.sin(lat_rads) * math.sin(max_lat_rads)))

        maxLat = math.degrees(max_lat_rads)
        maxLong = math.degrees(max_lon_rads)

        return (maxLat,maxLong)
    



In [18]:
CO_Springs_lat_and_lon = (38.833881,-104.821365)
get_bounds_in_area(CO_Springs_lat_and_lon[0],CO_Springs_lat_and_lon[1],5)

{'top right': (38.865669713496985, -104.78052884172236),
 'bottom right': (38.80207808250494, -104.78056531050585),
 'bottom left': (38.80207808250494, -104.86216468949414),
 'top left': (38.865669713496985, -104.86220115827764)}

In [60]:
def get_top_ten_segments_in_area(lat,lon,distance):
    area_sqaure = get_bounds_in_area(lat,lon,distance)
    bounds =  [area_sqaure["bottom left"][0],area_sqaure["bottom left"][1],area_sqaure["top right"][0],area_sqaure["top right"][1]]# array[Float] | The latitude and longitude for two points describing a rectangular boundary for the search: [southwest corner latitutde, southwest corner longitude, northeast corner latitude, northeast corner longitude]
    activity_type = "running" # String | Desired activity type. (optional)

    try: 
        # Explore segments
        api_response = api_instance.explore_segments(bounds, activity_type=activity_type)
        return api_response.to_dict()
    except ApiException as e:
        print("Exception when calling SegmentsApi->exploreSegments: %s\n" % e)
        return None

In [80]:
import json
import os.path
def save_segments_to_json_file(filename,dict):
    filename = "data/"+filename
    # check for file already there
    try:
        with open(filename,"r") as file:
            segment_data_dict = json.load(file)
    except Exception as e:
        print(e)
        segment_data_dict = None
    # append new segments to file if not already there
    if segment_data_dict:
        for segment in dict["segments"]:
            if segment not in segment_data_dict["segments"]:
                segment_data_dict["segments"].append(segment)
    else:
        segment_data_dict = dict
    json_object = json.dumps(segment_data_dict)
    # Writing to sample.json
    with open(filename, "w+") as outfile:
        outfile.write(json_object)


Saving a sample api call to json file

In [76]:
api_resp = get_top_ten_segments_in_area(34.096676,-117.719780,5)

'str' object has no attribute 'read'
None


In [87]:
api_resp = get_top_ten_segments_in_area(CO_Springs_lat_and_lon[0],CO_Springs_lat_and_lon[1],5)

In [ ]:
segment_data_file_name = "segment_data.json"
if api_resp:
    save_segments_to_json_file(segment_data_file_name,api_resp)
   

## Getting a list of Summary activies
Note: Summary Activities do not contain segment data, only detailed activities do. You cannot get multiple detailed activites at once.

In [99]:
api_instance = swagger_client.ActivitiesApi(swagger_client.ApiClient(configuration))


In [100]:
api_resp = api_instance.get_logged_in_athlete_activities()

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Tue, 13 Dec 2022 01:25:54 GMT', 'Via': '1.1 linkerd, 1.1 linkerd, 1.1 4e9b9147b7b73ce3eecd6a9f981a924a.cloudfront.net (CloudFront)', 'Vary': 'Origin', 'Server': 'nginx/1.21.3', 'Status': '401 Unauthorized', 'X-Request-Id': 'ac1591a7-ef41-4d07-b95b-22b51f1f628f', 'Cache-Control': 'no-cache', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'X-Frame-Options': 'DENY', 'X-XSS-Protection': '1; mode=block', 'X-RateLimit-Limit': '100,1000', 'X-RateLimit-Usage': '3,18', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-Permitted-Cross-Domain-Policies': 'none', 'X-Cache': 'Error from cloudfront', 'X-Amz-Cf-Pop': 'ORD58-P4', 'X-Amz-Cf-Id': 'QYowGToa6ijs9wslcP_m8Y7aQqqV7vX0NF7IEcn0O0LOaV6tqekDww=='})
HTTP response body: {"message":"Authorization Error","errors":[{"resource"

In [ ]:
def save_activities_to_json_file(filename,dict):
    filename = "data/"+filename
    # check for file already there
    try:
        with open(filename,"r") as file:
            segment_data_dict = json.load(file)
    except Exception as e:
        print(e)
        segment_data_dict = None
    # append new segments to file if not already there
    if segment_data_dict:
        for segment in dict["segments"]:
            if segment not in segment_data_dict["segments"]:
                segment_data_dict["segments"].append(segment)
    else:
        segment_data_dict = dict
    json_object = json.dumps(segment_data_dict)
    # Writing to sample.json
    with open(filename, "w+") as outfile:
        outfile.write(json_object)